In [1]:
import scipy

In [2]:
import glob

In [3]:
import cv2

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import numpy as np
import scipy.io as sio

In [6]:
path = '/Volumes/Samsung_T5/Oyla Dropbox/External/Annotations/RHD_House_data_August_14_20_46_23//oyla_output_R_fs_dus_spatialMedian_pclOutliers/'
file = 'oyla_0001'

In [7]:
imgRgb = plt.imread(path+'/rgb_jpg/'+file+'.jpg')[:100,:100,:]
imgDepth = cv2.imread(path+'/dist_png/'+file+'.png',cv2.IMREAD_UNCHANGED)[:100,:100]

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Samsung_T5/Oyla Dropbox/External/Annotations/RHD_House_data_August_14_20_46_23//oyla_output_R_fs_dus_spatialMedian_pclOutliers//rgb_jpg/oyla_0001.jpg'

In [8]:
imgIsNoise = (imgDepth == 0 );

NameError: name 'imgDepth' is not defined

In [9]:
maxImgAbsDepth = 50000
imgDepth = imgDepth / maxImgAbsDepth;
imgDepth[imgDepth > 1] = 1;

NameError: name 'imgDepth' is not defined

In [10]:
(H,W) = imgDepth.shape

NameError: name 'imgDepth' is not defined

In [11]:
numPix = H * W;

In [12]:
indsM = np.arange(numPix).reshape(imgDepth.shape[::-1]).transpose()

In [13]:
knownValMask =  (imgDepth != 0)

In [14]:
knownValMask

array([[False,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [15]:
gray = cv2.cvtColor(imgRgb,cv2.COLOR_RGB2GRAY)

In [77]:
winRad = 1;

L = -1;
absImgNdx = 0;
cols = np.zeros((numPix * (2*winRad+1)**2,1));
rows = np.zeros((numPix * (2*winRad+1)**2,1));
vals = np.zeros((numPix * (2*winRad+1)**2,1));
gvals = np.zeros((1, (2*winRad+1)**2));

In [78]:
rows.shape

(90000, 1)

In [79]:
numPix*9

90000

In [80]:
for j in range(W):
    for i in range(H):
        absImgNdx = absImgNdx + 1;
        nWin = -1; # Counts the number of points in the current window.
        for ii in range(max(0, i-winRad) , min(i+winRad, H-1)+1):
            for jj in range(max(0, j-winRad) , min(j+winRad, W-1)+1):
                #print(ii,jj)
                if ii == i and jj == j:
                    continue;
                L += 1
                nWin = nWin+1;
                rows[L] = absImgNdx;
                cols[L] = indsM[ii,jj]
                gvals[0][nWin] = gray[ii, jj];
        #            end
        #end
        #print(nWin)
        curVal = gray[i, j];
        gvals[0][nWin+1] = curVal;
        c_var = np.mean((gvals[0][:nWin+1]-np.mean(gvals[0][:nWin+1]))**2);
        #print(c_var)
        csig = c_var*0.6;
        mgv = np.min((gvals[0][:nWin+1]-curVal)**2);
        if csig < (-mgv/np.log(0.01)):
            csig=-mgv/np.log(0.01);
        #end
      
        if csig < 0.000002:
            csig = 0.000002;
        #end

        gvals[0][:nWin+1] = np.exp(-(gvals[0][:nWin+1]-curVal)**2/csig);
        gvals[0][:nWin+1] = gvals[0][:nWin+1] / np.sum(gvals[0][:nWin+1]);
        vals[L-nWin : L+1,0] = -gvals[0][:nWin+1];

#         #Now the self-reference (along the diagonal).
        L = L + 1;
        rows[L] = absImgNdx;
        cols[L] = absImgNdx;
        vals[L] = 1;#sum(gvals(1:nWin));
        #end
    #end

In [82]:
vals[0]

array([-0.99999939])

In [58]:
vals = vals[:L+1]
cols = cols[:L+1]
rows = rows[:L+1]
#A = sparse(rows, cols, vals, numPix, numPix);

In [68]:
A = scipy.sparse.csr_matrix((np.squeeze(vals), (np.squeeze(rows), np.squeeze(cols))), shape=(numPix+1, numPix+1))

In [67]:
np.max(rows)

10000.0

In [23]:
W

100

In [69]:
rows = np.arange(W*H)
cols = np.arange(W*H)
vals = knownValMask.transpose().reshape(-1) * 1;
#G = sparse(rows, cols, vals, numPix, numPix);
G = scipy.sparse.csr_matrix((np.squeeze(vals), (np.squeeze(rows), np.squeeze(cols))), shape=(numPix+1, numPix+1))

In [75]:
(A).diagonal()

array([0.        , 0.99999969, 0.75228895, ..., 0.8       , 0.99999252,
       1.        ])

In [71]:
from scipy.sparse.linalg import inv

In [ ]:
np.linalg.pinv(A.todense()+G.todense())

In [26]:
A.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]])